In [1]:
!pip install accelerate peft datasets bitsandbytes evaluate -q
!pip install rouge_score
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 23.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=485f1f8d7c354640c82d5917f350c74e3ebc5bae5900fb9a3055ec269b1be44c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00


# **Import library and load module**

In [2]:
import torch
import numpy as np
import evaluate
import json
import pandas as pd
import shutil
from IPython.display import FileLink
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from transformers import (
    LlamaForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    default_data_collator,
)
from datasets import load_dataset, Dataset, DatasetDict
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from peft import PeftModel, PeftConfig
import os

In [3]:
from huggingface_hub import login

hf_token = os.getenv("HF_TOKEN")

login(token=hf_token)

In [11]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [5]:
model_id = "meta-llama/Llama-3.2-1B"

In [ ]:
model_id = "/kaggle/input/llama-continued-pretrain-model/transformers/default/1/llama-continued-pretrain-model"

In [ ]:
model_id = "/kaggle/input/llama3.2-qa-model/transformers/default/1"

In [6]:
model_id = "/kaggle/input/llama-finetune-all/transformers/default/1/llama_finetune_model"

In [4]:
model_id = "/kaggle/input/llama-vnd-finetuned-model-epoch-2/transformers/default/1"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
# Thiết lập pad_token và padding side
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = LlamaForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},
)

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.config.use_cache = False

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [6]:
def collate_fn(batch):
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id

    input_ids = [item["input_ids"] for item in batch]
    attention_masks = [item["attention_mask"] for item in batch]
    labels = [item["labels"] for item in batch]

    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_masks = pad_sequence(attention_masks, batch_first=True, padding_value=0)
    labels = pad_sequence(labels, batch_first=True, padding_value=-100)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_masks,
        "labels": labels
    }

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Li

# **Continue Pretrain**

In [ ]:
dataset = load_dataset("vukhai248/vietnamese_news_16k", split="train")
dataset = dataset.select(range(2000))

In [ ]:
def preprocess_text(text):
    return text.replace("\n", " ").strip()

In [ ]:
def tokenize_fn(text, max_length=2048):
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id
        
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        truncation=True,
        max_length=max_length,
        padding=False,
        return_tensors="pt"
    )
    input_ids = encoding["input_ids"].squeeze(0)
    attention_mask = encoding["attention_mask"].squeeze(0)
    labels = input_ids.clone()
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
texts = dataset["content"]
tokenized_data = [tokenize_fn(text) for text in texts]
train_loader = DataLoader(
    tokenized_data,
    batch_size=2,
    shuffle=True,
    collate_fn=collate_fn
)

In [ ]:
def train(model, train_loader, device, n_epochs=3, lr=1e-5):
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    for epoch in range(n_epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}", leave=True)
        for batch in progress_bar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            
            # Forward pass with checkpointing
            def forward_with_checkpoint(*args, **kwargs):
                return torch.utils.checkpoint.checkpoint(
                    lambda *args, **kwargs: model(*args, **kwargs),
                    *args,
                    **kwargs,
                    use_reentrant=False
                )

            outputs = forward_with_checkpoint(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs.loss

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{n_epochs} - Loss: {avg_loss:.4f}")
train(model, train_loader, device)

In [ ]:
output_dir = "/kaggle/working/llama3-qlora-continued-pretrain"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Pretrained model saved at:", output_dir)

In [ ]:

# Đường dẫn tới thư mục chứa mô hình
model_dir = '/kaggle/working/llama3-qlora-continued-pretrain'

# Tạo file nén .zip từ thư mục
zip_name = 'llama3-qlora-continued-pretrain'  # Tên file nén (không cần đuôi .zip)
shutil.make_archive(zip_name, 'zip', model_dir)

# Tạo liên kết tải xuống cho file .zip
display(FileLink(f'{zip_name}.zip'))

# **Finetune with QA dataset**

In [ ]:
# model_id = "/kaggle/input/llama-qa-partially-finetune/transformers/default/1/llama-qa-finetune"

In [ ]:
with open("/kaggle/input/vietnamese-squad/train-v2.0-translated.json", "r", encoding="utf-8") as f:
    vi_squad_data = json.load(f)

In [ ]:
train_data = vi_squad_data[:10000]

# Lấy 2k mẫu tiếp theo cho validation
val_data = vi_squad_data[10000:12000]

print(f"📦 Train samples: {len(train_data)}")
print(f"📦 Validation samples: {len(val_data)}")

In [ ]:
print(f"📦 Train samples: {train_data[0]}")

In [ ]:
train_dataset = Dataset.from_list([{"context": item[0], "question": item[1], "answer": item[2]} for item in train_data])
val_dataset = Dataset.from_list([{"context": item[0], "question": item[1], "answer": item[2]} for item in val_data])

dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": val_dataset
})

In [ ]:
print(f"1. {dataset_dict['train'][0]}")
print(f"2. {dataset_dict['train'][0]['context']}")
print(f"3. {dataset_dict['train'][0]['question']}")
print(f"4. {dataset_dict['train'][0]['answer']}")

In [ ]:
def preprocess_qa(example, max_length=512):
    context = example["context"]
    question = example["question"]
    answer = example["answer"]
    prompt = f"### Đây là dạng câu hỏi và trả lời dựa trên nội dung ###\n\nCâu hỏi: {question}\n\nNội dung: {context}\n\nTrả lời:"
    completion = f" {answer}"
    
    prompt_ids = tokenizer(prompt, add_special_tokens=False).input_ids
    completion_ids = tokenizer(completion, add_special_tokens=False).input_ids

    input_ids = prompt_ids + completion_ids

    labels = [-100] * len(prompt_ids) + completion_ids

    if len(input_ids) > max_length:
        input_ids = input_ids[:max_length]
        labels = labels[:max_length]

    attention_mask = [1] * len(input_ids)
    padding_length = max_length - len(input_ids)
    
    input_ids = input_ids + [tokenizer.pad_token_id] * padding_length
    attention_mask = attention_mask + [0] * padding_length
    labels = labels + [-100] * padding_length
    
    return {
        "input_ids": torch.tensor(input_ids),
        "attention_mask": torch.tensor(attention_mask),
        "labels": torch.tensor(labels)
    }

tokenized_dataset = dataset_dict.map(preprocess_qa)

tokenized_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"]
)

In [ ]:
train_loader = DataLoader(
    tokenized_dataset["train"], batch_size=4, shuffle=True, collate_fn=collate_fn
)

val_loader = DataLoader(
    tokenized_dataset["test"], batch_size=4, shuffle=False, collate_fn=collate_fn
)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
n_epochs = 2

for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}", leave=True)

    for batch in progress_bar:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1}/{n_epochs} - Train Loss: {avg_loss:.4f}")

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"📊 Validation Loss: {avg_val_loss:.4f}")


In [ ]:
output_dir = "/kaggle/working/llama3-qlora-qa-finetune"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Pretrained model saved at:", output_dir)

In [ ]:
model_dir = '/kaggle/working/llama3-qlora-qa-finetune'

# Tạo file nén .zip từ thư mục
zip_name = 'llama3-qlora-qa-finetune'  # Tên file nén (không cần đuôi .zip)
shutil.make_archive(zip_name, 'zip', model_dir)

# Tạo liên kết tải xuống cho file .zip
display(FileLink(f'{zip_name}.zip'))

# **Finetune with BBC News**

In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
data_path = "/kaggle/input/data-llama-finetune/bbc_data_llama_finetune.json"

full_dataset = load_dataset("json", data_files=data_path, split="train")
dataset = full_dataset.train_test_split(test_size=0.1)

In [ ]:
def tokenize(example, max_length=2048):
    prompt = f"### Đây là dạng tóm tắt văn bản tin tức với độ dài tóm tắt đầu ra khoảng 150 từ:  ### Lệnh:\n{example['prompt']}\n\nTóm tắt:\n"
    summary = example["summary"]
    prompt_ids = tokenizer.encode(prompt, add_special_tokens=False)
    summary_ids = tokenizer.encode(summary, add_special_tokens=False)
    
    # Kiểm tra tổng số token
    total_length = len(prompt_ids) + len(summary_ids)
    if total_length > max_length:
        overflow = total_length - max_length
        if overflow < len(prompt_ids):
            prompt_ids = prompt_ids[:-overflow]
        else:
            prompt_ids = []
    input_ids = prompt_ids + summary_ids
    attention_mask = [1] * len(input_ids)
    labels = [-100] * len(prompt_ids) + summary_ids

    padding_length = max_length - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + [tokenizer.pad_token_id] * padding_length
        attention_mask = attention_mask + [0] * padding_length
        labels = labels + [-100] * padding_length
    else:
        input_ids = input_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]
    
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }
tokenized_dataset = dataset.map(tokenize, batched=False, fn_kwargs={"max_length": 2048})
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

collator = default_data_collator

train_loader = DataLoader(
    tokenized_dataset["train"],
    batch_size=2,
    shuffle=True,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    tokenized_dataset["test"],
    batch_size=2,
    shuffle=False,
    collate_fn=collate_fn
)


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
n_epochs = 3

for epoch in range(n_epochs):
    print(f"Epoch {epoch+1}/{n_epochs}")
    model.train()
    pbar = tqdm(train_loader)
    total_loss = 0

    for batch in pbar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        pbar.set_description(f"Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1} completed — Avg loss: {avg_loss:.4f}")

In [ ]:
output_dir = "/kaggle/working/llama3-qlora-bbc-finetuned"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Finetuned model saved at:", output_dir)

In [ ]:
model_dir = '/kaggle/working/llama3-qlora-bbc-finetuned'

# Tạo file nén .zip từ thư mục
zip_name = 'llama3-qlora-bbc-finetune'  # Tên file nén (không cần đuôi .zip)
shutil.make_archive(zip_name, 'zip', model_dir)

# Tạo liên kết tải xuống cho file .zip
display(FileLink(f'{zip_name}.zip'))

# **Finetune with donvanban**

In [ ]:
#model_id = "/kaggle/input/llama_finetune_v1.3/transformers/default/1/data_llama_finetune_v1.3"

In [ ]:
data_path = "/kaggle/input/data-llama-finetune/vi_data_llama_finetune.json"

full_dataset = load_dataset("json", data_files=data_path, split="train")
dataset = full_dataset.train_test_split(test_size=0.1)

In [ ]:
def tokenize(example, max_length=2048):
    prompt = f"### Đây là dạng tóm tắt văn bản tin tức với độ dài tóm tắt đầu ra khoảng 150 từ:  ### Lệnh:\n{example['prompt']}\n\nTóm tắt:\n"
    summary = example["summary"]
    
    # Mã hóa riêng prompt và summary (không thêm special tokens)
    prompt_ids = tokenizer.encode(prompt, add_special_tokens=False)
    summary_ids = tokenizer.encode(summary, add_special_tokens=False)
    
    # Kiểm tra tổng số token
    total_length = len(prompt_ids) + len(summary_ids)
    if total_length > max_length:
        overflow = total_length - max_length
        # Ưu tiên giữ lại phần summary; cắt bớt prompt
        if overflow < len(prompt_ids):
            prompt_ids = prompt_ids[:-overflow]
        else:
            prompt_ids = []  # Nếu quá tràn, bỏ hết prompt
    
    # Nối prompt và summary
    input_ids = prompt_ids + summary_ids
    attention_mask = [1] * len(input_ids)
    
    # Tạo labels: phần prompt được mask bằng -100, phần summary giữ nguyên token IDs
    labels = [-100] * len(prompt_ids) + summary_ids
    
    # Padding tất cả các trường về độ dài max_length
    padding_length = max_length - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + [tokenizer.pad_token_id] * padding_length
        attention_mask = attention_mask + [0] * padding_length
        labels = labels + [-100] * padding_length
    else:
        # Nếu quá dài, cắt bớt (nên không xảy ra nhờ truncation ở trên)
        input_ids = input_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]
    
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }
tokenized_dataset = dataset.map(tokenize, batched=False, fn_kwargs={"max_length": 2048})
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

collator = default_data_collator

train_loader = DataLoader(
    tokenized_dataset["train"],
    batch_size=2,
    shuffle=True,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    tokenized_dataset["test"],
    batch_size=2,
    shuffle=False,
    collate_fn=collate_fn
)


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
n_epochs = 3

for epoch in range(n_epochs):
    print(f"Epoch {epoch+1}/{n_epochs}")
    model.train()
    pbar = tqdm(train_loader)
    total_loss = 0

    for batch in pbar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        pbar.set_description(f"Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1} completed — Avg loss: {avg_loss:.4f}")

In [ ]:
output_dir = "/kaggle/working/llama3-qlora-finetuned-donvanban"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Finetuned model saved at:", output_dir)

In [ ]:
model_dir = '/kaggle/working/llama3-qlora-finetuned-donvanban'

# Tạo file nén .zip từ thư mục
zip_name = 'llama3-qlora-donvanban-finetune'  # Tên file nén (không cần đuôi .zip)
shutil.make_archive(zip_name, 'zip', model_dir)

# Tạo liên kết tải xuống cho file .zip
display(FileLink(f'{zip_name}.zip'))

# **Finetune with vietnamese-news-data**

In [8]:
df = pd.read_csv('/kaggle/input/vietnamese-news-data/Dataset_articles.csv')
df = df.sample(n=min(3000, len(df)), random_state=42).reset_index(drop=True)
# Làm sạch dữ liệu
df['Title'] = df['Title'].fillna('').astype(str)
df['Contents'] = df['Contents'].fillna('').astype(str)
df['Summary'] = df['Summary'].fillna('').astype(str)

# Gộp Title + Contents để tạo prompt
df['FullContent'] = df['Title'] + '. ' + df['Contents']

In [9]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

In [10]:
train_df.head(3)

,Unnamed: 0,URL,Title,Summary,Contents,Date,Author(s),Category,Tags,FullContent
642,293864,https://laodong.vn/bong-da-quoc-te/nhan-dinh-c...,Nhận định chung kết Asian Cup 2019: Nhật Bản v...,"Soi kèo, nhận định, dự đoán tỉ số, đội hình dự...","Sau gần 1 tháng tranh tài, vòng chung kết Asia...","Thứ sáu, 01/02/2019 13:06 (GMT+7)",Phương Anh,Thể thao,"['Nhật Bản', 'Nhật Bản vs Qatar', 'Soi kèo Nhậ...",Nhận định chung kết Asian Cup 2019: Nhật Bản v...
700,281563,https://laodong.vn/bong-da/doi-tuyen-viet-nam-...,Đội tuyển Việt Nam có phải cách ly khi dự Vòng...,"Đội tuyển Việt Nam có thể không phải cách ly, ...",Tại cuộc họp trực tuyến với đại diện Liên đoàn...,"Thứ ba, 23/02/2021 14:10 (GMT+7)",PHẠM ĐÌNH,Thể thao,"['Đội tuyển Việt Nam', 'World Cup 2022', 'bảng...",Đội tuyển Việt Nam có phải cách ly khi dự Vòng...
226,11201,https://laodong.vn/thoi-su/bat-dau-quy-trinh-n...,"Bắt đầu quy trình nhân sự chủ chốt Nhà nước, Q...","Ngày 30.3, Quốc hội bắt đầu tiến hành quy trìn...","Theo chương trình kỳ họp thứ 11, Quốc hội khoá...","Thứ ba, 30/03/2021 10:48 (GMT+7)",Đặng Chung - Đông Phương,Thời sự,"['Thủ tướng chính phủ', 'Chủ tịch Quốc hội', '...","Bắt đầu quy trình nhân sự chủ chốt Nhà nước, Q..."


In [11]:
def tokenize(example, max_length=2048):
    # Tạo prompt theo định dạng chuẩn
    prompt = (
        "### Đây là dạng tóm tắt văn bản tin tức với độ dài tóm tắt đầu ra khoảng 150 từ:\n"
        "### Lệnh:\n"
        "Bạn là một trợ lý tóm tắt văn bản. Hãy cung cấp bản tóm tắt ngắn gọn và chính xác trong 150 chữ cho bài viết sau.\n"
        f"Bài viết: {example['FullContent']}\n\nTóm tắt:\n"
    )
    summary = example["Summary"]

    # Token hóa prompt và summary riêng biệt, không thêm special tokens
    prompt_ids = tokenizer.encode(prompt, add_special_tokens=False)
    summary_ids = tokenizer.encode(summary, add_special_tokens=False)

    # Cắt prompt nếu quá dài để đảm bảo tổng token <= max_length
    total_length = len(prompt_ids) + len(summary_ids)
    if total_length > max_length:
        overflow = total_length - max_length
        prompt_ids = prompt_ids[:-overflow] if overflow < len(prompt_ids) else []

    # Tạo chuỗi đầu vào và attention mask
    input_ids = prompt_ids + summary_ids
    attention_mask = [1] * len(input_ids)

    # Label: che phần prompt bằng -100 (không tính loss)
    labels = [-100] * len(prompt_ids) + summary_ids

    # Padding đến max_length
    pad_len = max_length - len(input_ids)
    if pad_len > 0:
        input_ids = [tokenizer.pad_token_id] * pad_len + input_ids
        attention_mask = [0] * pad_len + attention_mask
        labels = [-100] * pad_len + labels


    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(test_df)

# Tokenize từng phần
tokenized_train = train_dataset.map(tokenize, fn_kwargs={"max_length": 2048})
tokenized_val = val_dataset.map(tokenize, fn_kwargs={"max_length": 2048})

# Set format để dùng với PyTorch DataLoader
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])



Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [12]:
train_loader = DataLoader(
    tokenized_train,
    batch_size=2,
    shuffle=True,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    tokenized_val,
    batch_size=2,
    shuffle=False,
    collate_fn=collate_fn
)

In [13]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Li

In [16]:
def save_and_link_model_zip(model_dir, zip_name=None):
    if zip_name is None:
        zip_name = os.path.basename(os.path.normpath(model_dir))
    
    zip_path = shutil.make_archive(zip_name, 'zip', model_dir)
    print(f"✅ Model saved and zipped to: {zip_path}")
    display(FileLink(f"{zip_name}.zip"))

In [17]:
n_epochs = 2

for epoch in range(n_epochs):
    print(f"Epoch {epoch+1}/{n_epochs}")
    model.train()
    pbar = tqdm(train_loader)
    total_loss = 0

    for batch in pbar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        pbar.set_description(f"Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1} completed — Avg loss: {avg_loss:.4f}")

Epoch 1/2


  0%|          | 0/1200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Loss: 1.5735: 100%|██████████| 1200/1200 [3:29:45<00:00, 10.49s/it] 


✅ Epoch 1 completed — Avg loss: 1.6352
Epoch 2/2


Loss: 1.1533: 100%|██████████| 1200/1200 [3:30:01<00:00, 10.50s/it] 

✅ Epoch 2 completed — Avg loss: 1.5741


In [22]:
output_dir = "./llama3-qlora-finetuned-vnd"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./llama3-qlora-finetuned-vnd/tokenizer_config.json',
 './llama3-qlora-finetuned-vnd/special_tokens_map.json',
 './llama3-qlora-finetuned-vnd/tokenizer.json')

In [23]:
model_dir = '/kaggle/working/llama3-qlora-finetuned-vnd'

# Tạo file nén .zip từ thư mục
zip_name = 'llama3-qlora-vnd-finetune'  # Tên file nén (không cần đuôi .zip)
shutil.make_archive(zip_name, 'zip', model_dir)

# Tạo liên kết tải xuống cho file .zip
display(FileLink(f'{zip_name}.zip'))

/kaggle/working/llama3-qlora-vnd-finetune.zip

In [21]:
def evaluate_model(model, val_loader, tokenizer, device, num_samples=3, max_eval_samples=300):
    model.eval()
    predictions, references, samples_to_print = [], [], []

    rouge = evaluate.load("rouge")
    bert_score = evaluate.load("bertscore")

    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token

    num_processed = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"]

            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=128,
                num_beams=2,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
            )

            # Decode output and clean
            raw_generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            cleaned_generated_texts = [extract_summary(text) for text in raw_generated_texts]

            # Decode labels (loại -100)
            label_texts = [
                tokenizer.decode([tid for tid in label.tolist() if tid != -100], skip_special_tokens=True)
                for label in labels
            ]

            predictions.extend(cleaned_generated_texts)
            references.extend(label_texts)

            # Lưu lại vài mẫu để in ra
            raw_inputs = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
            for src, ref, pred in zip(raw_inputs, label_texts, cleaned_generated_texts):
                if len(samples_to_print) < num_samples:
                    samples_to_print.append((src, ref, pred))

            num_processed += len(input_ids)
            if num_processed >= max_eval_samples:
                break

    # ROUGE
    rouge_results = rouge.compute(predictions=predictions, references=references, use_stemmer=True)

    # BERTScore trên 300 mẫu
    bert_results = bert_score.compute(predictions=predictions[:300], references=references[:300], lang="vi")
    bert_f1 = np.mean(bert_results["f1"])

    return rouge_results, bert_f1, samples_to_print


def print_evaluation_results(rouge_results, bert_f1, samples_to_print):
    print("\n📊 ROUGE Scores:")
    for key in rouge_results:
        print(f"{key}: {rouge_results[key]:.4f}")

    print(f"\n📈 BERTScore F1 (trên 300 mẫu): {bert_f1:.4f}")

    print("\n📝 Sample Results:")
    for i, (src, ref, pred) in enumerate(samples_to_print):
        print(f"\n--- Sample {i+1} ---")
        print(f"[Prompt]    {src[:300]}...")
        print(f"[Reference] {ref}")
        print(f"[Generated] {pred}")


In [22]:
rouge_results, bert_f1, samples = evaluate_model(model, val_loader, tokenizer, device)

Evaluating:  50%|████▉     | 149/300 [59:32<1:00:19, 23.97s/it]


In [23]:
print_evaluation_results(rouge_results, bert_f1, samples)


📊 ROUGE Scores:
rouge1: 0.4547
rouge2: 0.4494
rougeL: 0.4541
rougeLsum: 0.4541

📈 BERTScore F1 (trên 300 mẫu): 0.8032

📝 Sample Results:

--- Sample 1 ---
[Prompt]    ### Đây là dạng tóm tắt văn bản tin tức với độ dài tóm tắt đầu ra khoảng 150 từ:
### Lệnh:
Bạn là một trợ lý tóm tắt văn bản. Hãy cung cấp bản tóm tắt ngắn gọn và chính xác trong 150 chữ cho bài viết sau.
Bài viết: TP.Hồ Chí Minh: Mệt mỏi với bảng giá đất. UBND TPHCM vừa giao Sở Tài nguyên và Môi tr...
[Reference] Bảng giá đất của thành phố Hồ Chí Minh vừa được ban hành chưa được bao lâu nhưng đang có nhiều ý kiến trái chiều sau khi Sở Tài nguyên và Môi trường có văn bản hướng dẫn và dự kiến sắp tới đây lại có sự thay đổi.
[Generated] Bảng giá đất của thành phố Hồ Chí Minh vừa được ban hành chưa được bao lâu nhưng đang có nhiều ý kiến trái chiều sau khi Sở Tài nguyên và Môi trường có văn bản hướng dẫn và dự kiến sắp tới đây lại có sự thay đổi. Tuy nhiên, theo luật sư Hoàng Thu, Đoàn Luật sư TPHCM, trong quyết định ban hà